In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
from scipy.integrate import odeint
import scipy.signal
import scipy.interpolate

In [ ]:
import sys
sys.path.append('../')
import utils as ut
import os
import analysis_stn_data.plotting_functions as plotter
from definitions import DATA_PATH, SAVE_PATH_DATA, SAVE_PATH_FIGURES
import scipy

### Van der Pol oscillator

Is defined as
$$ 
\frac{d^2 x}{dt^2} - \mu (1 - x^2) \frac{dx}{dt} + x = 0
$$

Can be rewritten by defining $y=\dot x$ as: 

\begin{align}
\dot x &= y \\
\dot y &= \mu (1 - x^2)y + x 
\end{align}

The period of the relaxation oscillation is approx by $T = \mu (3 - ln 2)$

In [ ]:
def get_period(mu): 
    return mu * (3 - np.log(2))

In [ ]:
mu = 3.

def van_der_pol_oscillator_deriv(x, t):
    x1 = x[1]
    x2 = mu * (1.0 - x[0] ** 2.0) * x[1] - x[0]
    return np.array([x1, x2])

In [ ]:
dt = 0.001
fs = 1/dt
t_end = 400
t = np.arange(0, t_end, dt)
x = odeint(van_der_pol_oscillator_deriv, [0.1, .5], t)
x1 = x[:, 0]
x2 = x[:, 1]

In [ ]:
plt.figure(figsize=(15, 5))
start_idx = 5000
plt.plot(t[start_idx:], x1[start_idx:], label='x')
#plt.plot(t[start_idx:], x2[start_idx:], label='y')
plt.legend();

In [ ]:
T = get_period(mu)
print('frequency is f={}'.format(1/T))

### plot the phase distribution

In [ ]:
analytic_signal = scipy.signal.hilbert(x1)

# get the instantaneous phase over time
phase = np.angle(analytic_signal)

# sum up and average the phase vectors on the unit circle
vectors = np.exp(1j * phase)
circular_mean_vector = np.mean(vectors)

In [ ]:
plt.hist(phase, bins='auto');

### Calculate HFO amplitude to show its connection to the phase

In [ ]:
f, psd = scipy.signal.welch(x1, fs=fs, nperseg=1000)

In [ ]:
plt.plot(f[:50], psd[:50], 'o-')
plt.grid('on')

In [ ]:
f, tx, sxx = scipy.signal.spectrogram(x1, fs=fs, nperseg=1000)

In [ ]:
f[:10]

In [ ]:
a = 15000 
o = 40000

amp = sxx[2:10, :].sum(axis=0)
f = scipy.interpolate.interp1d(tx, amp)
amp = f(t[10000:-10000])

In [ ]:
amp_mat = np.tile(amp[a:o], (10, 1))

In [ ]:
plt.plot(amp)

In [ ]:
import matplotlib
fontsize = 20
plt.figure(figsize=(15, 7))
plt.subplot(211)
plt.title('Van der Pol oscillator, $\mu=3$', fontsize=fontsize)
plt.plot(t[a:o], x1[a:o], label='van der Pol oscillation, $\mu=3$')
#plt.xlim([0, t_end])
plt.xticks([], [])
plt.yticks(np.linspace(-2, 2, 3), np.linspace(-2, 2, 3))

plt.subplot(212)
plt.title('HFO amplitude phase-locked to the slow oscillation', fontsize=fontsize)
plt.imshow(amp_mat, norm=matplotlib.colors.PowerNorm(1), aspect=200, interpolation='None')
plt.xticks([], [])
plt.xlabel('time', fontsize=20)
#plt.ylabel('HFO power', fontsize=fontsize)
plt.yticks([], [])


#plt.xticks(np.linspace(0, amp_mat.shape[1]-1, 11), np.linspace(0, 40, 10, dtype=int))
plt.tight_layout();

In [ ]:
fig = plt.figure(figsize=(15, 3))
plt.plot(t[a:o], x1[a:o], label='van der Pol oscillation, $\mu=3$')
plt.xlabel('time ', fontsize=fontsize)
plt.xticks([], [])
# Make the y-axis label, ticks and tick labels match the line color.
plt.ylabel('slow oscillation', color='C0', fontsize=fontsize)
plt.tick_params('y', colors='C0')
plt.yticks(np.linspace(-2, 2, 3), np.linspace(-2, 2, 3))


plt.twinx()
plt.plot(t[a:o], amp[a:o], color='C1', label='higher frequencies')
plt.ylabel('power', color='C1', fontsize=fontsize)
plt.tick_params('y', colors='C1')
power_amp = 0.5
plt.ylim([0, power_amp])
plt.yticks(np.linspace(0, power_amp, 3), np.linspace(0, power_amp, 3))
plt.legend(prop={'size': 15})

# plot phase markers 
b = 15.9
for idx in range(3): 
    plt.axvline(x=b, color='C2', alpha=0.2)
    plt.axvline(x=b + 4.3, color='C2', alpha=0.2)
    b += 8.8

plt.title('Van der Pol oscillator with phase-locking in higher frequencies, $\mu=3$', fontsize=fontsize)
fig.tight_layout()
figure_path = os.path.join(SAVE_PATH_FIGURES, 'van_der_pol_illustration_poster.pdf')
plt.savefig(figure_path)